This project will clean and analyze data pulled from the German Ebay classifieds section.

In [45]:
import pandas as pd
import numpy as np

In [46]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [ ]:
autos.info()

In the cell below I clean up the column names to make them easier to read, and to turn them into snakecase, being it is the preffered python notation.

In [47]:
autos.columns
def column_cleaning(string):
    string = string.replace('dateCrawled', 'date_crawled')
    string = string.replace('offerType', 'offer_type')
    string = string.replace('vehicleType', 'vehicle_type')
    string = string.replace('yearOfRegistration', 'registration_year')
    string = string.replace('powerPS', 'power_ps')
    string = string.replace('monthOfRegistration','registration_month')
    string = string.replace('fuelType','fuel_type')
    string = string.replace('notRepairedDamage','unrepaired_damage')
    string = string.replace('dateCreated','ad_created')
    string = string.replace('nrOfPictures','num_pictures')
    string = string.replace('postalCode','postal_code')
    string = string.replace('lastSeen','last_seen')
    return string
cleaned_columns = []
for row in autos.columns:
    cleaned = column_cleaning(row)
    cleaned_columns.append(cleaned)
autos.columns = cleaned_columns

In [48]:
autos.rename({'kilometer': 'odometer_km'}, axis=1, inplace=True)

In the cell below I am cleaning out the prices column by setting a null value to all prices that are over 500000 and under 1. I wanted to try and remove the largest section of outliers while still maintaing the data for the rest of the columns. There were actually over 10000 entires in the price column that were equal to 0.

In [ ]:
autos.loc[(autos['price'] < 1) | (autos['price'] > 500000), 'price'] = np.nan
autos['price'].value_counts(dropna=False)

In [ ]:
autos['date_crawled'].str[:10].describe()
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

The dates range from March 5, 2016 to April 7, 2016. the distribution of the range of these dates is even across all dates. The most frquent date shown is April 5, 2016.

In [ ]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index()
autos['ad_created'].str[:10].describe()

The dates here range from March 10, 2014 to April 7, 2016. The most frequently seen date is April 3 2016. 

In [ ]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index()
#autos['last_seen'].str[:10].describe()

The dates here range from March 5, 2016 to April 7, 2016. The 3 most common dates here are April 5 2016 through April 7 2016 which together represent 45% of all of the dates here. The most seen date is April 6, 2016 which is seen in 21% of entries.

In [ ]:
autos['registration_year'].describe()

Many of the entries here are dates that are completely out of the ordinary. The lowest date is 1000 and the highest date is 9999. The mean listed is 2004.

In [ ]:
autos.loc[(autos['registration_year'] < 1900) | (autos['registration_year'] > 2016), 'registration_year'] = np.NaN
autos['registration_year'].describe()

There were over 14000 entries where the car registration year was below year 1900 and above year 2016. I decided to remove those which matched those parameters and set them to null values, as I can be sure there wouldn't have been any cars registered in those years sold in this time frame.

In [ ]:
autos['registration_year'].value_counts(normalize=True)

Here we can see the new minimum for the registration year is 1911, and the maximum is 2016. This ensures the data is far more accurate than before, as it is at least possible cars cold at this time would have been registered in those years. I kept the rest of those column's values since they may still be able to be used.

In [52]:
auto_brands = autos['brand'].unique()
auto_brands_dict = {}
for row in auto_brands:
    brand = autos[autos['brand'] == row]
    avg_price = brand['price'].mean()
    auto_brands_dict[row] = avg_price

the highest priced cars here are the porsche, land_rover, sonstige_autos, jaguar, jeep, mini, audi, mercedes_benz, and bmw which all average over $8000 per car. The lower priced cars are cars like Mitsubishi, Ford, and Fiat.

In [ ]:
auto_brands = autos['brand'].unique()
auto_brands_miles = {}
for row in auto_brands:
    brand = autos[autos['brand'] == row]
    avg_miles = brand['odometer_km'].mean()
    auto_brands_miles[row] = avg_miles

miles_series = pd.Series(auto_brands_miles)
price_series = pd.Series(auto_brands_dict)

df = pd.DataFrame(price_series, columns=['mean_price'])
df['mean_miles'] = miles_series
df.sort_values(by='mean_price', ascending=False)

There seems to be very littel correlation between the average number of miles on a car by brand and the price it sold for. I would venture a guess that the type of car matters more to most people than the number of miles on it when it comes to luxry cars such as Porsche, Audi, BMW, and Merceded Benz. 